## 6-1. 들어가며

### 쏘카에서 해결하고자 하는 것

___

우리는 쏘카의 수리 결과 처리를 위해 메모를 분석하고 있습니다. 지금까지 말뭉치로부터 형태소 분석까지 진행이 되었는데요. 이후 어떤 과정을 거쳐야 우리가 원하는 목표에 도달할 수 있을까요?

토큰화를 마친 결과물을 가지고 어떻게 의미 있는 정보를 찾아 메모의 카테고리 분류를 수행할 수 있을지 더 나아가 봅시다.

Q1. 지난 노드에서 알아본 내용 중에 꼭 간직해야 할 키워드는 어떤 것들이 있나요? 기억을 되살리며 적어보세요~!

예시답안

자연어처리, 말뭉치, 형태소 분석, 불용어, 토큰화, Subword, KoNLPy 등등. 기억이 안나신다면 어서 가서 슬쩍 보고 오세요!

### 학습 목표

___

-   토큰화된 결과물을 벡터로 변환할 수 있다.
-   word2vec을 훈련시키고 이용할 수 있다.
-   쏘카 메모 데이터를 활용하여 의미있는 정보를 찾아낼 수 있다.

### 학습 목차

___

1.  word2vec / FastText
2.  키워드 추출/분류
3.  시각화

### 준비물

___

지난 노드에서 사용한 폴더가 사라졌다면 다시 폴더를 만들어 주세요.

```
$ mkdir -p ~/aiffel/socar_memo/data
```

`gensim` 패키지의 warning을 해결하기 위해 아래 패키지를 추가로 설치해 주세요. 필수적인 것은 아닙니다. 아래 패키지를 설치하지 않으면 코드 동작 중에 warning이 등장할 수 있습니다.

```
$ pip install python-Levenshtein
```

클라우드 환경에서는 미리 준비된 데이터를 연결해 줍시다. 데이터에 대한 설명은 노드 진행 중에 볼 수 있습니다.  
(이전 노드에서 연결한 이력이 있다면 'File Exists'와 같은 오류 메세지가 나타날 수 있습니다. 그러나 이전 노드와 다른 데이터를 일부 사용하므로 꼭 연결하세요!)

```
$ ln -s ~/data/* ~/aiffel/socar_memo/data
```

## 6-2. word2vec

### Word embedding이란?

___

Word embedding이란 비정형화된 텍스트를 컴퓨터가 이해할 수 있도록 숫자로 바꿔줌으로써 사람의 언어를 컴퓨터의 언어로 번역하는 것을 의미해요. 가장 간단하게 생각하면 단어를 모두 정수(1, 2, 3...)로 변환하는 방법이 있겠네요! 그러나 이렇게 하면 단어 사전이 늘어지는 만큼 정수도 무한히 커지게 되는 문제가 생겨요.

이 방법의 대안으로는 원핫 인코딩이라는 방법을 사용할 수도 있는데요. 아래 그림처럼 표현하고자 하는 단어의 인덱스 값만 1로 하고, 나머지 인덱스는 전부 0으로 표현되는 벡터로 나타내는 방법이에요. 이렇게 벡터 또는 행렬의 값이 대부분 0으로 표현되는 것을 희소 표현(sparse representation)이라고 합니다. 그러나 이 방법은 차원의 저주에 빠지기 쉽고, 각 단어간의 유사성을 표현할 수 없다는 단점이 있어요. 또, 단어 집합의 크기만큼 0이 차지하는 공간이 커지는 한계가 있죠.

![content img](https://d3s0tskafalll9.cloudfront.net/media/original_images/so-4-p-1-1.png)

이러한 단점을 해결하기 위해 분산 표현(distributed representation)을 적용할 수 있습니다. 아래 그림처럼 다차원 공간에 벡터화하는 방법인데요. 원핫 인코딩의 희소 표현과 달리 밀집된 벡터로 표현할 수 있는 방법이에요. 원핫 인코딩에서는 각 차원이 분리된 형태로 표현되었지만, 분산 표현에서는 저차원에 여러 단어를 분산하여 표현하기 때문에 단어간 유사도를 계산할 수 있게 됩니다. 분산 표현을 사용하는 대표적인 예로 `word2vec`이 있어요.

-   [워드투벡터](https://wikidocs.net/22660)

### word2vec

___

word2vec이란 단어를 벡터로 변환하는 방법 중 하나로 어떤 분포 가설을 가정하고 그에 맞춰 표현한 분산 표현을 따르는 방법이에요. 기존 방법인 NNLM, RNNLM과 달리 빠른 속도로 분산 표현을 얻을 수 있어 크게 주목을 받았습니다.

word2vec에는 `CBOW`와 `skip-gram` 두 가지 학습 방법이 있습니다. 아래 그림처럼 두 방법의 가장 큰 차이는 학습 방법이에요. `CBOW`는 window 단어들을 기준으로 하나의 중심 단어를 예측하는 방법이고, `skip-gram`은 하나의 중심 단어를 기준으로 window 단어들을 예측합니다. 전반적인 논문들의 결과를 통해 `skip-gram`이 `CBOW`보다 더 좋은 성능을 보인다고 알려져 있습니다.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/so-4-p-1-3.max-800x600.png)

-   [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
-   [Distributed Representations of Words and Phrases and their Compositionality](https://papers.nips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf)

### 토큰화한 데이터를 word2vec으로 표현하기

___

앞에서 다룬 데이터 전처리 기법 중 토큰화를 적용하여 word2vec의 학습 데이터를 만들어 보겠습니다. 지금부터는 쏘카 데이터를 사용하여 진행하도록 하겠습니다!

먼저 사용할 라이브러리를 불러옵니다.

In [ ]:
import os
from collections import Counter
import pandas as pd
import gensim
from konlpy.tag import Komoran
import sentencepiece as spm
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic')
%config InlineBackend.figure_format = 'retina'

PATH = os.getenv('HOME') + '/aiffel/socar_memo/data'

print('슝=3')

미리 준비한 파일에는 `car_inspect_info.tsv`파일이 있습니다. 쏘카의 수리 내역을 일부 저장한 것입니다. 파일을 확인해 봅시다.

확장자가 `tsv`인 파일은 `csv`파일과 비슷하게 열 수 있습니다. comma를 구분자로 쓰는 `csv`([Comma-separated values](https://en.wikipedia.org/wiki/Comma-separated_values))파일 형식과 비슷하게 `tsv`([Tab-separated values](https://en.wikipedia.org/wiki/Tab-separated_values))는 tab을 구분자로 사용하기 때문입니다.

In [ ]:
memo_data = pd.read_csv(PATH+'/car_inspect_info.tsv', sep='\t', lineterminator='\r')
memo_data.head()

쏘카 데이터의 모습은 아래 그림과 같으며 총 273,118개의 행으로 이루어져 있습니다. 각 행은 6개의 열로 이루어집니다. 열은 `inspect_id`, `car_id`, `inspect_at`, `inspect_type`, `price`, `description`이고, 의미는 다음과 같습니다.

-   inspect\_id : 수리요청 고유 번호
-   car\_id : 자동차 고유 번호
-   inspect\_at : 수리요청 발생 시간
-   inspect\_type : 수리요청 카테고리
-   price : 수리 비용
-   description : 수리 관련 메모

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/so-4-p-1-4.max-800x600.png)

메모를 분석함에 있어서 필요한 열만 추리를 작업을 진행해 보도록 하겠습니다.

앞으로 `inspect_id`, `car_id`, `price`는 사용하지 않을 예정이니 지우겠습니다.

In [ ]:
# drop columns
memo_data = memo_data.drop(columns=['inspect_id', 'car_id', 'price'])

데이터가 들어가지 않은 항목은 없는지 확인해 볼까요?

In [ ]:
memo_data[memo_data.isna().any(axis=1)]

이 데이터를 모두 삭제해 줍니다.

In [ ]:
memo_data = memo_data.dropna() # drop nan row
memo_data.head()

이 다음으로 `description` 열에 해당하는 문장을 형태소 분석기를 통하여 토큰화 하겠습니다. `to_token` 함수를 통하여 문장에 포함된 특수문자 등을 공백으로 변환해주는 전처리를 진행하고, 형태소 분석기의 입력으로 넣어줍니다.

이 결과를 `morph_description`이라는 리스트에 모아서 원래 데이터 프레임 가장 마지막 열에 붙여 주었습니다.

`komoran`은 공백 문장('')을 입력을 받으면 오류를 발생시키기 때문에 try-except구문을 사용해 줍니다.

In [ ]:
komoran = Komoran()

def to_token(df, col):
    '''
    make sentence to subword sentence
    '''
    morph_description = []
    for idx, row in df.iterrows():
        memo = row[col]
    
        try: #After preprocessing memo sentence, do morph analysis
            morph_out = komoran.morphs(memo.replace('_',' ').replace('-',' ').replace('/',' ').replace('ㄴ',' ').replace('#',' ').replace('=',' ').replace(')',' ) ').replace('(',' ( '))
            morph_description.append(' '.join(morph_out))

        except: #if memo column is empty, drop all of row
            df.drop(idx, inplace=True)

    df["subword_description"] = morph_description # add column
    return df

memo_data = to_token(memo_data, "description")
memo_data.head()

이제 `gensim`을 사용하기 위해 데이터를 약간 변형해 줍시다.

아래 그림과 같이 라인 by 라인으로 된 1차원 리스트로 입력을 넣어주는 것이 아니라 문장이 토큰화된 형태로 입력해야 합니다. 즉, 2차원 리스트의 형태로 넣어주어야 합니다.

![content img](https://d3s0tskafalll9.cloudfront.net/media/original_images/so-4-p-1-5.png)

아래 코드를 실행해 학습 데이터를 정제해 주세요!

In [ ]:
w2v_train = list(memo_data['subword_description'])
w2v_train = [line.split() for line in w2v_train]

print(len(w2v_train))

### word2vec 옵션

___

학습 데이터 형태를 맞췄다면 이제 word2vec에서 제공하는 옵션에 대해 알아보도록 하겠습니다.

word2vec에서 제공하는 옵션은 [여기](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)에서 확인할 수 있습니다.

우리가 사용할 옵션은 아래 정보를 참고하세요!

-   vector\_size는 단어 벡터를 몇 차원으로 지정할 것인가에 대한 옵션입니다. 사람들이 주로 많이 사용하는 차원은 300입니다.
-   window는 앞서 언급한 word2vec의 주변 단어를 의미하고, 이 크기를 사용자가 지정할 수 있습니다.
-   epochs는 전체 학습 말뭉치를 몇 번 돌아서 볼 것인가를 의미합니다. 딥러닝에서 에폭과 같은 개념이라고 보시면 이해가 빠를 것 같습니다.
-   sg는 word2vec의 skip-gram의 모형을 쓸 지, 또는 CBOW를 사용할지 결정하는 옵션입니다. 1로 세팅하게 되면 skip-gram을 사용하는 것이고, 0으로 설정하게 되면 CBOW를 사용하는 것 입니다.
-   min\_count는 토큰이 되기위한 최소 빈도수를 지정하는 옵션입니다. 이 값을 1로 지정하게 되면 말뭉치에서 출현한 모든 단어에 대하여 단어 벡터를 만들 수 있습니다. 기본 값은 5입니다.

### word2vec 학습 및 사용하기

___

이제 word2vec을 사용해 학습시켜 보도록 하겠습니다.

In [ ]:
model = gensim.models.word2vec.Word2Vec(w2v_train, vector_size=300, window=5, min_count=1, epochs=10)
# save Word2Vec
model.save(PATH+'/vec300_w2v.model')
# load Word2Vec
model = gensim.models.Word2Vec.load(PATH+'/vec300_w2v.model')

print('슝=3')

학습이 완료된 벡터를 사용하여 유사도를 표현할 수 있습니다. `타이어`와 유사한 단어는 무엇인지 확인해 볼게요.

만약 단어의 출력 개수를 조정하고 싶다면 `topn`을 수정하여 작성하면 됩니다.

In [ ]:
# check word most similarity
model.wv.most_similar('타이어', topn=10)

이제 `엔진`과 가장 유사한 단어는 무엇인지 알아볼까요? 직접 작성해 보세요~!

In [ ]:
# Write your code
# [[Your Code]]

예시코드

```
model.wv.most_similar('엔진', topn=10)

```

word2vec의 결과를 통해 단어 벡터를 사용하면, 이와 관련된 유사한 단어를 얻을 수 있습니다. 하지만 상위 10개의 단어 중 하위에 위치한 단어는 유사성이 부족한 것을 볼 수 있네요. 이 부분을 보완하기 위해 top 10의 유사도 값을 모두 더하여 평균내고, 이 평균값을 기준(Threshold)로 정하여 기준보다 상위에 위치한 값을 사용하도록 할 수 있어요.

In [ ]:
def exclusion(word_list):
    count = sum([score for _, score in word_list])
    avg = count / len(word_list)
  
    up_word = []
    for w, s in word_list:
        if s > avg:
            up_word.append(w)
  
    return up_word

print('슝=3')

이 함수는 다음 단계에 사용되니 파악만 하고 넘어가요~!

## 6-3. FastText



FastText
___
단어를 벡터로 만드는 또 다른 방법으로는 Facebook의 FastText가 있어요.

-   [FastText][1]

[1]:https://fasttext.cc/

우리가 쓰는 gensim에도 FastText가 포함되어 있습니다.

-   [gensim.models.fasttext][2]

[2]:https://radimrehurek.com/gensim/models/fasttext.html



Q2. Facebook Open source인 FastText와 gensim의 FastText는 어떤 차이가 있을까요? (바로 위 링크에 힌트가 숨겨져 있습니다!)

예시답안

This module contains a fast native C implementation of fastText with Python interfaces. It is not only a wrapper around Facebook’s implementation.

This module supports loading models trained with Facebook’s fastText implementation. It also supports continuing training from such models.

FastText도 word2Vec이 나온 이후에 등장했기 때문에 word2vec의 확장판이라고 보시면 됩니다.

이 둘의 차이점으로는 word2vec은 단어(토큰)을 쪼갤 수 없는 개념으로 보는 반면, FastText는 하나의 단어를 N-gram 단위로 분절할 수 있다고 봅니다.

예제를 통해서 FastText의 N-gram과 학습 방법에 대해서 알아볼게요.

단어 `쏘카아이펠`의 경우 N을 3으로 두고 분절한 결과를 보면 아래와 같이 총 6개의 결과가 나오게 됩니다. 6개의 결과에는 단어의 시작 `<`과 그 뒤 음절 2개의 결합인 `<쏘카`가 있고, 단어의 끝 `>`과 그 앞 음절 2개의 결합인 `이펠>`이 포함됩니다. 또 마지막으로 단어 전체에 시작`<`과 `>`를 붙인 `<쏘카아이펠>`이 포함됩니다. 이렇게 구한 단어 분절 결과를 모두 벡터화에 사용합니다.

```
# N = 3 인 경우
단어 : 쏘카아이펠
분절 결과 : <쏘카, 쏘카아, 카아이, 아이펠, 이펠>, <쏘카아이펠> (총 6개)
```

실제 FastText를 사용할 때는 사용자가 최소/최대 N의 범위를 지정할 수 있어요. 기본으로 지정된 N의 범위는 최소 3, 최대 6입니다. 이 범위를 적용하여 `쏘카아이펠`을 다시 한번 분절해 볼게요. 그 결과 15개의 분절 결과를 얻게 되네요.

```
# N = 3 ~ 6 인 경우
단어 : 쏘카아이펠
분절 결과 : <쏘카, 쏘카아, 카아이, 아이펠, 이펠>, <쏘카아, 쏘카아이, 카아이펠, 아이펠>, <쏘카아이, 쏘카아이펠, 카아이펠>, <쏘카아이펠, 쏘카아이펠>, <쏘카아이펠> (총 15개)
```

이렇게 FastText에서는 단어 분절까지 고려하게 되면서, word2vec에서 얻을 수 없던 강점을 가지게 됩니다. 이런 강점을 통해 OOV(out-of-vocabulary)에서도 강건하다는 특징을 가지게 되죠. 단어 임베딩을 함에 있어 N-gram까지 고려하게 되면서, 사전에 학습되지 않은 단어에도 N-gram의 조합으로 단어 벡터값을 얻을 수 있어요.

예를들어 FastText 모델이 `자연어처리`라는 단어를 학습하지 않았어도 `자연어`와 `처리`에 대한 학습은 이루어 졌다고 가정해 볼게요. 기존 word2vec을 이용할 때 학습하지 못한 단어가 나타나면 에러가 나겠지만, FastText를 이용하면 `자연어`와 `처리`를 조합하여 `자연어처리`에 대한 벡터를 얻을 수 있습니다.

이런 특징으로 인해 상대적으로 FastText는 노이즈가 심한 말뭉치에 강건하다는 평을 듣습니다.

아래 그림에서 왼쪽에 나열된 결과는 word2vec을 이용한 결과이고, 오른쪽에 나열된 결과는 FastText를 이용한 결과예요. 두 단어 벡터 모델 모두 동일한 데이터로 학습되었는데요. word2vec과 FastText의 결과를 비교해보면 FastText가 입력 단어 `타이어`가 포함된(혹은 더 연관된) 단어를 많이 내보내는 것을 확인할 수 있네요.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/so-4-p-2-1.max-800x600.png)

아래 참고자료를 통해 FastText에 대해 좀 더 알아보세요!

-   [패스트텍스트(FastText)][3]

[3]:https://wikidocs.net/22883

## 6-4. 쏘카 메모 데이터

앞서 확인한 쏘카 메모 데이터에는 총 45개의 카테고리가 `inspect_type`컬럼에 정의되어 있습니다. 전체 행은 254,142개에 이릅니다. 그러나, 몇 개의 카테고리에는 세부적으로 나눌 수 있는 카테고리가 존재해요. 대표적으로 `일반수리` 카테고리에 해당합니다. `일반수리` 카테고리는 52,094개의 행으로 이루어져 있습니다.

그래서 이번 스텝에서는 `일반수리` 카테고리에 해당하는 메모 데이터를 다시 세부적인 수준으로 분류하는 과정을 순차적으로 나열해보고, 문제에 대한 해결 방안을 알아볼게요. 앞으로 진행하는 방법이 카테고리 분류에 있어 절대적인 정답은 아니고 해결할 수 있는 방법 중 하나라는 점을 잊지 마세요~!

### 데이터 준비

___

먼저 필요한 데이터를 준비해 보겠습니다.

자동차 사전이 필요하겠죠?

In [ ]:
# load car dictionary
def load_car_dic():
    car_dic_file = open(PATH+'/carDic.txt','r')
    car_dic = set(car_dic_file.read().split('\n'))
    return car_dic

car_dic = load_car_dic()
print(len(car_dic)) #1433

쏘카 메모 데이터도 다시 불러올게요. 데이터 원본 중에 필요한 컬럼만 남기고 nan이 들어 있는 행도 삭제해 줍니다.

In [ ]:
memo_data = pd.read_csv(PATH+'/car_inspect_info.tsv', sep='\t', lineterminator='\r')
memo_data = memo_data.drop(columns=['inspect_id', 'car_id', 'price']) # drop columns
memo_data = memo_data.dropna() # drop nan row
memo_data.head()

문장을 토큰화 할 때 형태소 분석기 대신 앞서 언급한 subword 토크나이저인 SentencePiece를 사용하도록 하겠습니다.

형태소 분석기 대신 SentencePiece를 사용하는 이유는 아래처럼 데이터 전체가 깔끔하게 정제되지 않은 경우(모두 붙여쓴 경우 등) 형태소 분석기는 잘못된 결과를 내보낼 가능성이 높기 때문이에요. 따라서 지금부터는 SentencePiece를 사용하여 메모 카테고리 분석을 진행해 보도록 하겠습니다.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/so-4-p-3-1.max-800x600.png)

SentencePiece는 학습시켜 이용할 수 있기 때문에 새로운 단어 처리나 상황에 맞는 처리에 유리합니다. 그래서 일반적인 문장을 위해 만들어진 형태소 분석기보다 자동차에 관계된 문장에 딱 맞는 subword 토크나이저를 준비할 수 있죠.

이제 학습에 사용될 데이터를 준비해 볼게요. 뉴스 기사 데이터 `KCC150_100k.txt`와 쏘카 메모 데이터 중 `description` 열을 모두 더해 사용하겠습니다. 학습 데이터를 이렇게 구성하는 이유는 쏘카 메모 데이터가 깔끔하게 전처리 되어있지 않았기 때문이에요. 노이즈가 심한 데이터로만 구성하여 토크나이저를 학습시키면 올바른 토큰화가 이루어지지 않을 것이라고 판단했습니다.

아래 그림에서 첫 번째 결과처럼 토큰화가 불완전한 모습을 볼 수 있겠죠. 두 데이터 셋을 섞어 올바르게 토큰화를 한다면 두 번재 결과처럼 좀 더 나은 결과물을 얻을 수 있습니다.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/so-4-p-3-2.max-800x600.png)

본격적으로 SentencePiece 학습 데이터를 구축하고 모델을 학습하여 토큰화에 적용하는 방법을 코드로 살펴보겠습니다.

SentencePiece를 학습하기 위해서는 Python의 string형태가 아니라 별도의 text파일로 저장되어야 합니다. 아래 코드를 통해 학습 파일을 만들어 봅시다.

In [ ]:
#sentenctePiece를 학습하기 위해서는 별도의 text 파일이 필요함
#따라서 이를 만드는 부분
with open(PATH+'/car_info_description.txt','w') as w, open(PATH+'/KCC150_100k.txt','r') as corpus:
    w.write('\n'.join(memo_data['description']))
    for idx, snt in enumerate(corpus.readlines()):
        w.write(snt)

print('슝=3')

이제 준비된 `car_info_description.txt` 파일을 이용하여 학습을 진행해 봅니다.

In [ ]:
#SentencePiece Train & load
spm.SentencePieceTrainer.Train(f'--input={PATH}/car_info_description.txt --model_prefix={PATH}/description --vocab_size=6000')
sp = spm.SentencePieceProcessor()
vocab_file = PATH+'/description.model'
sp.load(vocab_file)

print('슝=3')

이제 쏘카 메모 데이터의 `description` 열에 SentencePiece를 적용하여 토큰화하고 저장하겠습니다.

In [ ]:
def to_token(df, col):
    '''
    make sentence to subword sentence
    '''
    morph_description = []
    for _, row in df.iterrows():
        memo = row[col]
        sp_out = sp.encode_as_pieces(memo.replace('_',' ').replace('-',' ').replace('/',' ').replace('ㄴ',' ').replace('#',' ').replace('=',' ').replace(')',' ) ').replace('(',' ( '))
        morph_description.append(' '.join([token.replace('▁','') for token in sp_out]))

    df["subword_description"] = morph_description
    return df

memo_data = to_token(memo_data, "description")
memo_data.head()

이제 토큰화한 데이터를 이용해서 단어 벡터를 학습시켜 볼게요.

In [ ]:
# Word2Vec train data processing
w2v_train = list(memo_data['subword_description'])
w2v_train = [line.split() for line in w2v_train]

# Word2Vec train part
model = gensim.models.word2vec.Word2Vec(w2v_train, vector_size=300, window=5, min_count=1, epochs=10)

이렇게 학습된 모델을 얻었습니다. 이제 이 모델을 활용할 단계로 가볼까요~?

## 6-5. 키워드 추출

`일반수리` 카테고리의 메모 데이터를 훑어보면 하나의 카테고리로 묶을 수 있지만, 그에 해당하는 종류가 너무 많아서 이를 하나로 묶는 경우가 더 나은 경우를 확인할 수 있습니다.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/so-4-p-4-1.max-800x600.jpg)

\[자동차의 부품 종류\]

![image.png](attachment:image.png)

\[경고등의 종류\]

https://giwa-game.tistory.com/entry/%EC%9E%90%EB%8F%99%EC%B0%A8-%EA%B3%84%EA%B8%B0%ED%8C%90-%EA%B2%BD%EA%B3%A0%EB%93%B1-%EC%A2%85%EB%A5%98-%EC%A0%95%EB%A6%AC

위 경우들을 모두 하나의 매핑 테이블로 묶어서 처리하게 되면, 여러 개로 퍼져있는 단어들을 포괄적으로 묶을 수 있겠네요. 하지만 모든 경우를 수작업으로 묶기 어렵기 때문에 앞서 학습한 word2vec을 통해 유사한 단어들을 묶어 보도록 하겠습니다.

word2vec은 단어간의 유사도를 파악하기에 용이합니다. 아래 결과에서 하나의 단어에 대하여 유사한 단어들을 잘 표현함을 확인할 수 있습니다. 이러한 특징을 활용하나 매핑테이블을 구성해 보겠습니다.

In [ ]:
# check word most similarity
model.wv.most_similar('타이어', topn=10)

`타이어`와 유사한 단어들이 나오네요. 이제 `경고등`과 유사한 단어를 찾아볼까요? 직접 코드를 작성해 봅시다!

In [ ]:
# Write your code
# [[ Your Code ]]

예시 코드

```
model.wv.most_similar('경고등', topn=10)

```

만족스러운 결과이지만, 순위가 낮아질수록 상대적으로 입력 단어와 거리가 있는 것으로 보이네요. 이를 해결하기 위해 단어 벡터에서 유사도가 평균값 이상인 경우만 선택하도록 하겠습니다. 아래 함수를 이용해서 유사한 단어만 추려보도록 할게요.

In [ ]:
def exclusion(word_list):
    '''
    most_similar의 단어를 평균 이상만 추림
    '''
    count = sum([score for _, score in word_list])
    avg = count / len(word_list)
  
    up_word = []
    for w, s in word_list:
        if s > avg:
            up_word.append(w)
  
    return up_word

print('슝=3')

이제 이 함수를 이용해 봅시다.

In [ ]:
# 라이트, 전구 에서 나온 단어들 평균 이상만 저장
light = model.wv.most_similar('라이트', topn=20)
light_word = exclusion(light)
light = model.wv.most_similar('전구', topn=20)
light_word.extend(exclusion(light))

light_word.append('라이트')
light_word.append('전구')

light_word = set(light_word)

# 경고등도 평균 이상만 저장
error = model.wv.most_similar('경고등', topn=10)
error_word = exclusion(error)
error_word.append('경고등')

print(light_word)
print('')
print(error_word)

이제 이 단어들을 하나의 카테고리로 묶기 위해 매핑테이블을 작성해 볼게요.

매핑테이블이란 하나의 Key를 두고 여러 개의 Value를 Key에 연결시키는 것을 의미합니다. `딜리버리`와 `탁송비`라는 Key에 `탁송`라는 Value를 두어 `딜리버리`나 `탁송비`가 문장에서 출현하게 된다면 이를 `탁송`으로 매핑시켜주는 역할을 합니다. 이러한 방법은 하나의 카테고리에서 다양한 단어가 나올 수 있는 경우에 고정된 카테고리로 묶어주는 역할을 합니다. 아래 그림처럼 여러 개의 단어가 하나의 단어로 묶이게 되는 거죠.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/so-4-p-4-3.max-800x600.png)

In [ ]:
# 매핑 테이블 작성
mapping_table = {'내비게이션':'네비게이션', '내비':'네비게이션', '네비':'네비게이션', \
                  '딜리버리':'탁송', '탁송비':'탁송', \
                  '브레이크 오일':'브레이크액', '브레이크오일':'브레이크액', \
                  '라디오':'오디오'}

# "라이트" 관련 단어 매핑 테이블 추가
light_mapping = dict()
for word in light_word:
    light_mapping[word] = '라이트'

mapping_table.update(light_mapping)

# "경고등" 관련 단어 매핑 테이블 추가
error_mapping = dict()
for word in error_word:
    error_mapping[word] = '경고등'

mapping_table.update(error_mapping)

print(mapping_table)

이제 문장에서 카테고리가 얼마나 나왔는지 카운팅 해보도록 하겠습니다.

우선 데이터에서 나타난 전체 카테고리는 몇 개 인지 확인해 볼게요. 총 45개의 카테고리가 존재함을 확인할 수 있어요.

In [ ]:
#total 45 category
category = set(memo_data['inspect_type'])
print(len(category))

다음으로는 문장 내에서 카테고리 내에 공백이 들어간 경우를 고려하기 위해 `noun_phrase`함수를 만들어 줍니다. SentencePiece를 이용한 토큰화가 제대로 이루어지지 않을 가능성을 고려하여, 원래 문장을 어절 단위로 분절한 결과를 고려하여 카테고리를 찾는 `find_category`함수를 작성합니다.

In [ ]:
def noun_phrase(words):
    result = []

    for index, token in enumerate(words):
        if (token in car_dic) or (token in category):
            #명사구 check
            if (''.join(words[index : index +2]) in car_dic) or (''.join(words[index : index +2]) in category):
                result.append(''.join(words[index : index +2]))
            #명사구 X
            result.append(token)
    return result

def find_category(row):
    # word base
    line = row['description'].replace('_',' ').replace('-',' ').replace('/',' ').replace('ㄴ',' ').replace('#',' ').replace('=',' ').replace(')',' ) ').replace('(',' ( ')
    car_token = noun_phrase(line.upper().split())
    # subword base
    car_token.extend(noun_phrase(row['subword_description'].upper().split()))
    return set(car_token)

print('슝=3')

이제 `일반수리` 열에서 자동차 사전에 기반한 sub 카테고리가 얼마나 출현했는지 카운팅 해보겠습니다.

In [ ]:
subword_list = []
for _, row in memo_data.iterrows():
    if row['inspect_type'] == '일반수리':
        car_words = find_category(row) #find word in car_dic
    
        if len(car_words) >= 1:  #car_words more than one
            subword_list.extend(car_words)

subword_list = [subword for subword in subword_list if subword != '']
subword_counter = Counter(subword_list) #car_dic에 포함되는 단어 + category에 포함되는 단어
subword_sort = sorted(subword_counter.items(), key=lambda x:x[1], reverse=True)
subword_top20 = [(key, value) for key, value in subword_sort[:20]]

print(subword_top20)

결과를 보니 `전구`라는 단어가 가장 많이 출현했네요.

이제 이 카운팅 결괏값을 토대로 문장의 카테고리를 재분류하러 가볼까요?

## 6-6. 키워드 분류

앞에서 만든 결과를 토대로 이제 카테고리를 다시 분류해 볼 예정입니다.

먼저 다시 분류할 수 있는 함수를 작성할게요.

In [ ]:
def re_ranking(word_list):
    '''
    이전에 카테고리 카운팅된 subword_counter를 기준으로 Recategoring
    IF. 2개 이상의 카테고리 단어가 한 문장에서 등장할 경우, max 값 카테고리를 취함
    '''
    max_word = (None, -99)
    for word in word_list:
        # 단어가 mapping table에 포함되는 경우
        # mapping table Key word를 반환함
        if word in mapping_table:
            word = mapping_table.get(word)

        # 단어가 2개이상 선택되는 경우, 고빈도 word를 취함
        if subword_counter[word] > max_word[1]:
            max_word = (word, subword_counter[word])

    return max_word[0]

print('슝=3')

`re_ranking` 함수는 앞서 계산한 `subword_counter`를 기준으로 한 문장에서 2개 이상의 카테고리 단어가 출현할 경우, 결과 중에서 최댓값을 갖는 카테고리를 최종 카테고리로 선정하게 만드는 함수입니다. 즉, 여러 개의 단어가 있을 때, 다시 최종 카테고리를 정해주는 것이지요.

이제 이 함수를 이용해서 재분류 작업을 진행합니다.

In [ ]:
"""
find more best category from 일반수리
"""
category_count = 0
recategory_count = 0
recategory = []
for _, row in memo_data.iterrows():
    if row['inspect_type'] == '일반수리':
        carWords = find_category(row) #find word in carDic
        recat = re_ranking(carWords)
        recategory.append(recat)
    
        category_count += 1
        if recat != None:
            recategory_count += 1 

    else:
        recategory.append(None)

memo_data['recategory'] = recategory #append recategory columns

memo_data = memo_data[['inspect_at', 'inspect_type','recategory','description','subword_description']] #change columns order
memo_data.to_csv(PATH+'/car_inspect_info_NEW.tsv', mode='w')

memo_data.head()

최종적으로 `일반수리` 카테고리에 있어서 다시 sub 카테고리로 나눠진 경우는 `recategory` 열에 추가하여 `car_inspect_info_NEW.tsv` 파일로 저장하였습니다.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/so-4-p-5-1.max-800x600.png)

마지막으로 기존 `일반수리` 카테고리 52,094 행 중 19,928행을 sub 카테고리로 변환한 것을 확인해 볼까요?

In [ ]:
print('일반수리 카테고리 개수 : ', category_count)
print('일반수리 카테고리중 sub카테고리 변경 개수 : ', recategory_count)

이제 sub 카테고리의 개수를 카운팅 해보도록 하겠습니다.

In [ ]:
recategory_counter = Counter(recategory)
recategory_counter_sorted = sorted(recategory_counter.items(), key=lambda x:x[1], reverse=True)
recategory_top20 = [(key, value) for key, value in recategory_counter_sorted[:21]]
print(recategory_top20)

그 결과 라이트가 가장 높은 순위를 차지하는 것을 확인할 수 있네요. None이 최상단에 위치하는 이유는 `일반수리` 카테고리가 아닌 경우와 `일반수리` 카테고리에서 sub 카테고리를 못 찾은 결과가 포함되기 때문입니다.

## 6-7. 시각화

앞에서 나온 결과를 토대로 시간의 흐름에 따른 경향성을 파악해 보도록 하겠습니다.

우선 시간의 흐름을 보기 위해 날짜에 해당하는 데이터를 string에서 datetime으로 수정할게요.

In [ ]:
# 일반수리에 해당하는 row만 추림
normal = memo_data.loc[memo_data['inspect_type'] == '일반수리'].copy()

#pandas inspect_at column to datetime object
normal['dates'] = pd.to_datetime(normal['inspect_at'], format='%Y-%m-%d', errors='raise')

normal.head()

이제 이 데이터를 월별로 시각화 해보겠습니다.

In [ ]:
monthly = normal.groupby(pd.Grouper(key='dates', freq='M')).count()
monthly = monthly['inspect_at']
monthly = monthly.reset_index() #dates가 index인데, 이를 제거해줌(column으로 변경)
monthly['dates_str'] = monthly['dates'].apply(lambda x: str(x)[:10]) #datetime to str

plt.rcParams["figure.figsize"] = (8, 6)
monthly.plot(x='dates_str',y='inspect_at', color='#00B8FF', kind='bar') #Socar color

plt.xticks(rotation=45)
plt.title('Monthly Freq')
plt.xlabel('Month')
plt.ylabel('freq')

plt.savefig(PATH+'/Monthly_freq.png')
plt.show()

이제 sub 카테고리의 top 5 항목에 대해서만 발생 경향성을 파악해 보도록 하죠.

In [ ]:
# recategory freq Top 5
temp = pd.DataFrame()
for word, count in recategory_top20[1:6]:
    temp[word] = normal.groupby(pd.Grouper(key='dates', freq='M'))['recategory'].apply(lambda x: x[x == word].count())

temp.plot(marker='.')
plt.title('Monthly Freq')
plt.xlabel('Month')
plt.ylabel('freq')
plt.savefig(PATH+'/recategory_top5.png')
plt.show()

sub 카테고리 top 6~10은 어떨까요?

In [ ]:
# recategory freq Top 6-10
temp = pd.DataFrame()
for word, count in recategory_top20[6:11]:
    temp[word] = normal.groupby(pd.Grouper(key='dates', freq='M'))['recategory'].apply(lambda x: x[x == word].count())

temp.plot(marker='.')
plt.title('Monthly Freq')
plt.xlabel('Month')
plt.ylabel('freq')
plt.savefig(PATH+'/recategory_6_10.png')
plt.show()

이제 쏘카에서 수리되는 카테고리를 한 눈에 파악할 수 있게 되었습니다! 짝짝짝! 👏👏👏

## 6-8. 프로젝트

우리는 쏘카가 당면한 문제를 확인하고 어떻게 해결하기 위해 나아가는지 알아봤습니다. 어려운 건 없습니다! 한 단계 한 단계 밟아 나가면 되는 거였어요!

하지만 지금까지 해온 것은 기초 준비에 해당합니다. 앞으로 다른 시도를 해보면서 더 나은 방법은 있는지, 방법마다 어떤 차이가 있는지 알아보는 것이 중요합니다.

여러 시도 중에 좋은 방법이 발견될 때 비로소 쏘카의 문제를 해결하게 되겠죠.

먼저 프로젝트에서 사용할 주요 라이브러리 버전을 알아봅시다.

이제 그럼 어떤 시도를 하면 좋을지 알아봅시다.

In [ ]:
from importlib.metadata import version
import pandas
import gensim
import konlpy
import sentencepiece

print(pandas.__version__)
print(gensim.__version__)
print(konlpy.__version__)
print(version('sentencepiece'))

이제 그럼 어떤 시도를 하면 좋을지 알아봅시다.

### 여러 카테고리에 대해 시도해보자

___

우리가 사용한 카테고리는 `일반수리` 하나였어요. 하지만 쏘카 메모 데이터에는 다른 카테고리가 많이 있습니다.

이 다양한 카테고리를 어디까지 활용해야 더 나은 해법을 찾을 수 있을까요?

카테고리마다 개수를 세어보고 어느 카테고리를 더 분석하면 좋을지 계획을 세우고, 노드와 동일한 방법을 시도해 봅시다.

카테고리 조합에 따른 결과를 다양하게 분석해 볼 수록 좋은 결과를 얻을 수 있을 거예요!

### FastText 활용

___

노드에서는 `word2vec`을 사용했습니다. 하지만 앞서 이야기 했듯이 `FastText`를 사용하는 방법도 있습니다.

```
# sentences에 데이터를 전처리 해두었다고 가정합니다

model = gensim.models.fasttext.FastText(vector_size=300, window=5, min_count=1)
model.build_vocab(sentences)
model.train(sentences, total_examples=len(sentences), epochs=10)
```

`FastText`와 `word2vec` 두 방법을 사용한 결과를 비교해 봅시다.

### 응용해보자!

___

지금까지 우리가 얻은 맵핑테이블과 모델을 활용해보는 시간을 가져 봅시다. 노드에서는 `일반수리` 카테고리인 항목을 맵핑테이블과 모델을 활용해 메모에서 세부 카테고리를 추출하고, 시간에 따른 변화를 보았습니다.

이제 여러분의 간단한 프로그램 또는 함수를 작성하여 현실 문제를 개선하는 아이디어를 세우고 작성해 보세요.

예를 들면 공업사에서 전달된 문장 내 자동차 용어를 자동으로 치환해주는 함수, 카테고리 별로 전체 수리 횟수 추이를 파악하고 다음해 예산을 추정해보는 프로그램 등을 만들어볼 수 있겠네요!

![image.png](attachment:image.png)

In [ ]:
pip list

Package                       Version
----------------------------- ------------------
absl-py                       0.15.0
aiohttp                       3.8.1
aiosignal                     1.2.0
albumentations                1.1.0
antlr4-python3-runtime        4.8
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
asttokens                     2.0.5
astunparse                    1.6.3
async-generator               1.10
async-timeout                 4.0.2
attrs                         21.4.0
audioread                     2.1.9
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.6.0
black                         22.1.0
bleach                        4.1.0
branca                        0.4.2
brotlipy                      0.7.0
cachetools                    5.0.0
certifi                       2021.10.8
cffi                          1.15.0
charde

Note: you may need to restart the kernel to use updated packages.
